In [ ]:
library('Seurat')
library('ggplot2')
library(ggplot2)
library(ggalluvial)
library(ggpubr)
library(viridis)
library(tidyr)
library(stringr)
library(dplyr)

### Load un-annotated data and the classfier

Load your data that needs to be annotated along with the classifier made in the previous step

In [ ]:
combined <- readRDS("/path/to/your/data.rds")
scmap_classifier <- readRDS(file = "path/to/your/scmap_classifier.rds")
ref_stages <- c("E6.5","E6.75","E7.0","E7.25","E7.5","E7.75","E8.0","E8.25","E8.5") 
#Here you can choose the embryonic stages in the annotated dataset you need to be used

### Convert data into readable format by scmap

Convert your unannotated dataset from a seurat object to a SingleCellExperiment object

In [ ]:
sce <- as.SingleCellExperiment(x = combined)
rowData(sce)$feature_symbol <- rownames(sce)
scmapCluster_results <- scmapCluster(projection = sce, index_list = scmap_classifier[ref_stages],threshold = 0)

### Create a similarity matrix

scmap outputs a matrix of how similar an individual barcode is to its corresponding stage in the published annotations.

In [ ]:
siml <- scmapCluster_results$scmap_cluster_siml
siml <- as.data.frame(siml)
siml[is.na(siml)] = 0

### Add relevant columns and results to your dataset

From the similarity matrix, you can get the embryonic stage every barcode is assigned to. You can add that and the similarity scores to the metadata of your (now newly annotated) dataset.

In [ ]:
Emb_Stage <- colnames(siml)[max.col(siml,ties.method="first")]
Emb_Stage <- as.data.frame(Emb_Stage)
combined <- AddMetaData(object = combined, metadata = scmapCluster_results$combined_labs, col.name = "celltype")
Idents(combined) <- combined$celltype
combined@meta.data <- cbind(combined@meta.data, Emb_Stage)
combined@meta.data <- cbind(combined@meta.data, siml)

### Save your dataset

In [ ]:
saveRDS(combined, 'combined_annotated.rds')